In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('./result.xlsx')

In [3]:
df.head()

,date,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status
0,2017-01-01,周日,-,-,多云,无持续风向微风,372,严重
1,2017-01-02,周一,-,-,霾,无持续风向微风,361,严重
2,2017-01-03,周二,-,-,霾~雾,无持续风向微风,280,重度
3,2017-01-04,周三,9°,2°,小雨,无持续风向微风,193,中度
4,2017-01-05,周四,5°,1°,小雨,无持续风向微风,216,重度


In [4]:
# 设定索引为日期，方便按日期筛选
df.set_index('date', inplace=True)

In [5]:
df.head()

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status
date,,,,,,,
2017-01-01,周日,-,-,多云,无持续风向微风,372,严重
2017-01-02,周一,-,-,霾,无持续风向微风,361,严重
2017-01-03,周二,-,-,霾~雾,无持续风向微风,280,重度
2017-01-04,周三,9°,2°,小雨,无持续风向微风,193,中度
2017-01-05,周四,5°,1°,小雨,无持续风向微风,216,重度


In [6]:
# 替换掉温度的后缀°
df['min_temperature'] = df['min_temperature'].map(lambda x : int(x.replace('°','')) if x != '-' else np.nan)
df['max_temperature'] = df['max_temperature'].map(lambda x : int(x.replace('°','')) if x != '-' else np.nan)

In [7]:
df.dtypes

week                object
max_temperature    float64
min_temperature    float64
day_status          object
wind                object
aqi                  int64
aqi_status          object
dtype: object

## 1、使用单个label值查询数据¶
行或者列，都可以只传入单个值，实现精确匹配


In [8]:
# 得到单个值
df.loc['2017-01-04','aqi']

193

In [9]:
# 得到一个Series
df.loc['2017-01-04',['max_temperature','aqi']]


max_temperature      9
aqi                193
Name: 2017-01-04, dtype: object

## 2、使用值列表批量查询¶


In [10]:
# 得到Series
df.loc[['2018-01-03','2018-01-04','2018-01-05'], 'max_temperature']


date
2018-01-03    1.0
2018-01-04   -2.0
2018-01-05    0.0
Name: max_temperature, dtype: float64

In [11]:
# 得到DataFrame
df.loc[['2018-01-03','2018-01-04','2018-01-05'], ['max_temperature', 'min_temperature']]


,max_temperature,min_temperature
date,,
2018-01-03,1.0,-6.0
2018-01-04,-2.0,-10.0
2018-01-05,0.0,-8.0


## 3、使用数值区间进行范围查询

In [12]:
# 行index按区间
df.loc['2018-01-03':'2018-01-10', 'min_temperature']

date
2018-01-03    -6.0
2018-01-04   -10.0
2018-01-05    -8.0
2018-01-06    -6.0
2018-01-07    -5.0
2018-01-08    -5.0
2018-01-09    -5.0
2018-01-10    -8.0
Name: min_temperature, dtype: float64

In [13]:
# 列columns按区间
df.loc['2018-01-03', 'max_temperature':'aqi']

max_temperature          1
min_temperature         -6
day_status           小雪~大雪
wind               东北风1-2级
aqi                     67
Name: 2018-01-03, dtype: object

In [14]:
# 行和列都按区间查询
df.loc['2018-01-03':'2018-01-10', 'max_temperature':'aqi']

,max_temperature,min_temperature,day_status,wind,aqi
date,,,,,
2018-01-03,1.0,-6.0,小雪~大雪,东北风1-2级,67
2018-01-04,-2.0,-10.0,大雪~阴,东北风1-2级,61
2018-01-05,0.0,-8.0,多云~小雪,南风1-2级,92
2018-01-06,-2.0,-6.0,中雪~小雪,无持续风向微风,160
2018-01-07,0.0,-5.0,阴~多云,西风1-2级,177
2018-01-08,1.0,-5.0,多云~晴,西风4-5级,64
2018-01-09,2.0,-5.0,晴,西风3-4级,52
2018-01-10,4.0,-8.0,晴,西风3-4级,50


## 4、使用条件表达式查询

bool列表的长度得等于行数或者列数

In [15]:
df['min_temperature'] < 10 

date
2017-01-01    False
2017-01-02    False
2017-01-03    False
2017-01-04     True
2017-01-05     True
              ...  
2019-12-27     True
2019-12-28    False
2019-12-29    False
2019-12-30     True
2019-12-31     True
Name: min_temperature, Length: 1095, dtype: bool

### 复杂条件查询，查一下我心中的完美天气

注意，组合条件用&符号合并，每个条件判断都得带括号

In [16]:
# 查询最高温度小于30度，并且最低温度大于15度，并且是晴天，并且天气为优的数据
df[(df['max_temperature'] <= 30) & (df['min_temperature'] >= 15) & (df['day_status'] == '晴') & (df['aqi_status'] == '优')]

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status
date,,,,,,,
2018-09-07,周五,29.0,17.0,晴,北风1-2级,49,优


我哭，郑州的好天气，三年才一天。


## 5、调用函数查询

In [17]:
# 直接写lambda表达式
df.loc[lambda df : (df["max_temperature"]<=30) & (df["min_temperature"]>=15), :]

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status
date,,,,,,,
2017-04-14,周五,27.0,15.0,多云,西南风1-2级,117,轻度
2017-04-15,周六,29.0,17.0,多云~小雨,西风1-2级,125,轻度
2017-04-16,周日,24.0,15.0,小雨~多云,无持续风向微风,69,良
2017-04-17,周一,28.0,16.0,晴,西风3-4级,114,轻度
2017-04-23,周日,29.0,15.0,晴~多云,无持续风向微风,105,轻度
...,...,...,...,...,...,...,...
2019-09-27,周五,30.0,17.0,多云~晴,东南风2级,81,良
2019-09-28,周六,30.0,19.0,多云~晴,东南风2级,76,良
2019-10-02,周三,30.0,18.0,多云~晴,东南风2级,71,良


In [18]:
# 编写自己的函数，查询9月份，空气质量好的数据
def query_my_data(df):
    return df.index.str.startswith("2019-09") & (df["aqi"] <= 100)
df.loc[query_my_data, :]

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status
date,,,,,,,
2019-09-01,周日,32.0,20.0,多云,东南风2级,62,良
2019-09-02,周一,30.0,20.0,多云,东南风2级,53,良
2019-09-03,周二,30.0,19.0,多云,东南风2级,60,良
2019-09-04,周三,31.0,19.0,多云,东南风2级,80,良
2019-09-05,周四,32.0,20.0,晴,东南风2级,72,良
2019-09-06,周五,33.0,20.0,晴,东南风2级,68,良
2019-09-07,周六,34.0,21.0,晴,西南风2级,87,良
2019-09-08,周日,36.0,23.0,晴,东南风2级,88,良
2019-09-09,周一,34.0,23.0,多云,东南风2级,82,良


## 6、直接赋值的方法¶
实例：清理温度列，变成数字类型

In [19]:
df['dif'] = df['max_temperature'] - df['min_temperature']

In [20]:
df['dif'].head()

date
2017-01-01    NaN
2017-01-02    NaN
2017-01-03    NaN
2017-01-04    7.0
2017-01-05    4.0
Name: dif, dtype: float64

In [21]:
df.columns

Index(['week', 'max_temperature', 'min_temperature', 'day_status', 'wind',
       'aqi', 'aqi_status', 'dif'],
      dtype='object')

In [22]:
df.index

Index(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04', '2017-01-05',
       '2017-01-06', '2017-01-07', '2017-01-08', '2017-01-09', '2017-01-10',
       ...
       '2019-12-22', '2019-12-23', '2019-12-24', '2019-12-25', '2019-12-26',
       '2019-12-27', '2019-12-28', '2019-12-29', '2019-12-30', '2019-12-31'],
      dtype='object', name='date', length=1095)

## 7、df.apply方法
Apply a function along an axis of the DataFrame.

Objects passed to the function are Series objects whose index is either the DataFrame’s index (axis=0) or the DataFrame’s columns (axis=1).

实例：添加一列温度类型：

- 如果最高温度大于33度就是高温
- 低于-10度是低温
- 否则是常温

In [23]:
# apply
def get_temperature_type(df):
    if df['max_temperature'] >= 33:
        return '高温'
    elif df['min_temperature'] <= -10:
        return '低温'
    return '常温'

df['temp_type'] = df.apply(get_temperature_type, axis=1)

In [24]:
df['temp_type'].value_counts()

常温    909
高温    185
低温      1
Name: temp_type, dtype: int64

In [25]:
# apply 第二个方法
df.loc[:, 'temp_type2'] = df.apply(lambda x : '高温' if x['max_temperature'] >= 33 else '低温' if x['min_temperature'] <= -10 else '常温', axis=1)

In [26]:
df['temp_type2'].value_counts()


常温    909
高温    185
低温      1
Name: temp_type2, dtype: int64

## 8、df.assign方法
Assign new columns to a DataFrame.

Returns a new object with all original columns in addition to new ones.


实例：将温度从摄氏度变成华氏度


In [27]:
# assign 返回一个新对象 同时添加多列
df.assign(
    # 华氏摄氏度
    max_t = lambda x: x['max_temperature'] * 9 / 5 + 32,
    min_t = lambda x: x['min_temperature'] * 9 / 5 + 32,
)

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status,dif,temp_type,temp_type2,max_t,min_t
date,,,,,,,,,,,,
2017-01-01,周日,NaN,NaN,多云,无持续风向微风,372,严重,NaN,常温,常温,NaN,NaN
2017-01-02,周一,NaN,NaN,霾,无持续风向微风,361,严重,NaN,常温,常温,NaN,NaN
2017-01-03,周二,NaN,NaN,霾~雾,无持续风向微风,280,重度,NaN,常温,常温,NaN,NaN
2017-01-04,周三,9.0,2.0,小雨,无持续风向微风,193,中度,7.0,常温,常温,48.2,35.6
2017-01-05,周四,5.0,1.0,小雨,无持续风向微风,216,重度,4.0,常温,常温,41.0,33.8
...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,周五,14.0,-2.0,晴~多云,西南风2级,55,良,16.0,常温,常温,57.2,28.4
2019-12-28,周六,NaN,NaN,阴~多云,西南风2级,80,良,NaN,常温,常温,NaN,NaN
2019-12-29,周日,NaN,NaN,多云~晴,西南风2级,116,轻度,NaN,常温,常温,NaN,NaN


## 9、按条件选择分组分别赋值

按条件先选择数据，然后对这部分数据赋值新列
实例：高低温差大于10度，则认为温差大

In [28]:
# wen cha
# 先创建空列 可以不先创建
df['wencha_type'] = ''

In [29]:
# 筛选满足条件的行，然后赋值
df.loc[(df['max_temperature'] - df['min_temperature']) > 10, 'wencha_type'] = '温差大'
df.loc[(df['max_temperature'] - df['min_temperature']) <= 10, 'wencha_type'] = '温差小'

In [30]:
df['wencha_type'].value_counts()

温差小    600
温差大    455
        40
Name: wencha_type, dtype: int64

## 10 统计函数

### 10.1汇总类统计函数


In [31]:
# 展示数字列的统计结果
# count 计数
# mean  平均值
# std   标准差
# min   最小值
# max   最大值
# xx%   不同位置的百分比数值
# 50%    中位数
df.describe()

,max_temperature,min_temperature,aqi,dif
count,1055.000000,1055.000000,1095.000000,1055.000000
mean,21.766825,11.885308,104.319635,9.881517
std,10.458676,10.037459,60.200473,3.098661
min,-2.000000,-10.000000,20.000000,2.000000
25%,13.000000,3.000000,66.500000,8.000000
50%,23.000000,13.000000,86.000000,10.000000
75%,31.000000,21.000000,119.500000,12.000000
max,39.000000,29.000000,422.000000,20.000000


In [32]:
# 查看Series平均值
df['max_temperature'].mean()

21.766824644549764

In [33]:
# 最高温
df['max_temperature'].max()

39.0

In [34]:
# 最低温
df['max_temperature'].min()

-2.0

### 10.2唯一性去重和按值计数

#### 10.2.1唯一性去重

一般不用于数值类列，而是枚举，分类列

In [35]:
df.head(3)

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status,dif,temp_type,temp_type2,wencha_type
date,,,,,,,,,,,
2017-01-01,周日,NaN,NaN,多云,无持续风向微风,372,严重,NaN,常温,常温,
2017-01-02,周一,NaN,NaN,霾,无持续风向微风,361,严重,NaN,常温,常温,
2017-01-03,周二,NaN,NaN,霾~雾,无持续风向微风,280,重度,NaN,常温,常温,


In [36]:
df['day_status'].unique()

array(['多云', '霾', '霾~雾', '小雨', '阵雨~多云', '晴', '阴~多云', '多云~阴', '晴~多云',
       '多云~晴', '小雨~雨夹雪', '阴', '雨夹雪~多云', '阴~小雪', '阴~雨夹雪', '暴雪~小雪', '小雨~多云',
       '小雨~阴', '多云~小雨', '中雨~小雨', '阵雨', '阵雨~阴', '晴~阴', '多云~阵雨', '阴~阵雨',
       '小雨~大雨', '大雨~小雨', '晴~雷阵雨', '雷阵雨~阴', '雷阵雨~多云', '雷阵雨~中雨', '雷阵雨',
       '雷阵雨~阵雨', '大雨~阴', '小雨~中雨', '中雨', '阵雨~小雨', '阴~小雨', '多云~小雪', '小雪~阴',
       '小雪~大雪', '大雪~阴', '中雪~小雪', '小雪~中雪', '小雪~多云', '小雨~阵雨', '阵雨~中雨',
       '中雨~阵雨', '中雨~大雨', '大雨', '小雨~晴', '晴~小雨', '雨夹雪~阴', '中雨~雨夹雪', '小雪',
       '小雨~暴雨', '中雨~多云'], dtype=object)

In [37]:
df['wind'].unique()

array(['无持续风向微风', '西北风3-4级', '北风4-5级', '东北风3-4级', '西南风1-2级', '南风3-4级',
       '东北风4-5级', '西北风4-5级', '东南风1-2级', '南风1-2级', '东北风1-2级', '东风1-2级',
       '西风1-2级', '西南风3-4级', '西风3-4级', '北风3-4级', '东风3-4级', '北风1-2级',
       '西北风1-2级', '东南风3-4级', '西风4-5级', '西北风5-6级', '南风4-5级', '东北风5-6级',
       '西风5-6级', '西北风3级', '西南风2级', '东南风2级', '西北风2级', '西南风1级', '东北风2级',
       '东北风1级', '西北风1级', '东北风3级', '东南风1级', '西北风4级', '西南风3级', '东北风4级',
       '东南风3级', '西风3级', '西南风4级', '南风3级', '南风2级'], dtype=object)

In [38]:
df['aqi_status'].unique()

array(['严重', '重度', '中度', '轻度', '良', '优'], dtype=object)

#### 10.2.2按值计数

In [39]:
df['aqi_status'].value_counts()

良     628
轻度    224
优      76
重度     76
中度     71
严重     20
Name: aqi_status, dtype: int64

In [40]:
# 百分比显示 normalize
df['day_status'].value_counts(normalize=True)

多云        0.239269
晴         0.199087
晴~多云      0.094977
多云~晴      0.080365
多云~阴      0.062100
阴~多云      0.051142
小雨        0.050228
多云~小雨     0.027397
阴         0.025571
小雨~多云     0.023744
阴~小雨      0.014612
小雨~阴      0.013699
阵雨        0.011872
阵雨~多云     0.010046
多云~阵雨     0.009132
小雨~中雨     0.007306
中雨~小雨     0.007306
小雪~多云     0.005479
中雨        0.004566
多云~小雪     0.004566
小雨~大雨     0.004566
雷阵雨       0.003653
阵雨~小雨     0.003653
雷阵雨~中雨    0.002740
阴~阵雨      0.002740
阴~小雪      0.002740
阵雨~阴      0.002740
晴~雷阵雨     0.002740
小雨~阵雨     0.001826
晴~阴       0.001826
小雪~阴      0.001826
雨夹雪~多云    0.001826
阴~雨夹雪     0.001826
雷阵雨~阴     0.001826
小雨~雨夹雪    0.000913
霾~雾       0.000913
雷阵雨~阵雨    0.000913
小雨~暴雨     0.000913
小雪        0.000913
阵雨~中雨     0.000913
大雨        0.000913
霾         0.000913
暴雪~小雪     0.000913
大雨~阴      0.000913
小雪~大雪     0.000913
晴~小雨      0.000913
中雨~雨夹雪    0.000913
小雪~中雪     0.000913
中雨~阵雨     0.000913
雷阵雨~多云    0.000913
中雨~多云     0.000913
雨夹雪~阴     0.000913
中雪~小雪     0.

##  11、相关系数和协方差

用途（超级厉害）：

1. 两只股票，是不是同涨同跌？程度多大？正相关还是负相关？
2. 产品销量的波动，跟哪些因素正相关、负相关，程度有多大？

来自知乎，对于两个变量X、Y：

1. 协方差：***衡量同向反向程度\***，如果协方差为正，说明X，Y同向变化，协方差越大说明同向程度越高；如果协方差为负，说明X，Y反向运动，协方差越小说明反向程度越高。
2. 相关系数：***衡量相似度程度\***，当他们的相关系数为1时，说明两个变量变化时的正向相似度最大，当相关系数为－1时，说明两个变量变化的反向相似度最大


In [41]:
# 协方差矩阵
df.cov()

,max_temperature,min_temperature,aqi,dif
max_temperature,109.383908,100.266400,-219.976416,9.117508
min_temperature,100.266400,100.750590,-254.507379,-0.484190
aqi,-219.976416,-254.507379,3624.097009,34.530963
dif,9.117508,-0.484190,34.530963,9.601698


In [42]:
# 相关系数矩阵
df.corr()

,max_temperature,min_temperature,aqi,dif
max_temperature,1.000000,0.955113,-0.367766,0.281336
min_temperature,0.955113,1.000000,-0.443352,-0.015567
aqi,-0.367766,-0.443352,1.000000,0.194853
dif,0.281336,-0.015567,0.194853,1.000000


In [43]:
# 单独查看空气质量和最高温度的相关系数
df['aqi'].corr(df['max_temperature'])

-0.36776569069963444

In [44]:
df['aqi'].corr(df['min_temperature'])

-0.44335170726841794

In [45]:
# 空气质量和温差的相关系数
df['aqi'].corr(df['max_temperature'] - df['min_temperature'])

0.1948527230549141

## 12、数据清洗

Pandas使用这些函数处理缺失值：

- isnull和notnull：检测是否是空值，可用于df和series
- dropna：丢弃、删除缺失值
  - axis : 删除行还是列，{0 or ‘index’, 1 or ‘columns’}, default 0
  - how : 如果等于any则任何值为空都删除，如果等于all则所有值都为空才删除
  - inplace : 如果为True则修改当前df，否则返回新的df
- fillna：填充空值
  - value：用于填充的值，可以是单个值，或者字典（key是列名，value是值）
  - method : 等于ffill使用前一个不为空的值填充forword fill；等于bfill使用后一个不为空的值填充backword fill
  - axis : 按行还是列填充，{0 or ‘index’, 1 or ‘columns’}
  - inplace : 如果为True则修改当前df，否则返回新的df

### 12.1实例：特殊Excel的读取、清洗、处理

<img src="https://cdn.jsdelivr.net/gh/wfy-belief/PicGo-images//blog/20200818182415.png" style="zoom:50%;" />

### 步骤1：读取excel的时候，忽略前几个空行

In [46]:
tmp_data = pd.read_excel('./student_excel.xlsx', skiprows=2)

In [47]:
tmp_data

,Unnamed: 0,姓名,科目,分数
0,NaN,小明,语文,85.0
1,NaN,NaN,数学,80.0
2,NaN,NaN,英语,90.0
3,NaN,NaN,NaN,NaN
4,NaN,小王,语文,85.0
5,NaN,NaN,数学,NaN
6,NaN,NaN,英语,90.0
7,NaN,NaN,NaN,NaN
8,NaN,小刚,语文,85.0
9,NaN,NaN,数学,80.0


### 步骤2：检测空值

In [48]:
tmp_data.isnull()

,Unnamed: 0,姓名,科目,分数
0,True,False,False,False
1,True,True,False,False
2,True,True,False,False
3,True,True,True,True
4,True,False,False,False
5,True,True,False,True
6,True,True,False,False
7,True,True,True,True
8,True,False,False,False
9,True,True,False,False


In [49]:
tmp_data['分数'].isnull()

0     False
1     False
2     False
3      True
4     False
5      True
6     False
7      True
8     False
9     False
10    False
Name: 分数, dtype: bool

In [50]:
tmp_data.notnull()

,Unnamed: 0,姓名,科目,分数
0,False,True,True,True
1,False,False,True,True
2,False,False,True,True
3,False,False,False,False
4,False,True,True,True
5,False,False,True,False
6,False,False,True,True
7,False,False,False,False
8,False,True,True,True
9,False,False,True,True


In [51]:
# 筛选没有空分数的所有行
tmp_data.loc[tmp_data['分数'].notnull(), :]

,Unnamed: 0,姓名,科目,分数
0,NaN,小明,语文,85.0
1,NaN,NaN,数学,80.0
2,NaN,NaN,英语,90.0
4,NaN,小王,语文,85.0
6,NaN,NaN,英语,90.0
8,NaN,小刚,语文,85.0
9,NaN,NaN,数学,80.0
10,NaN,NaN,英语,90.0


### 步骤3：删除掉全是空值的列

In [52]:
# tmp_data.drop(axis=1, how='all', inplace=True)
tmp_data.dropna(axis='columns', how='all', inplace=True)

In [53]:
tmp_data

,姓名,科目,分数
0,小明,语文,85.0
1,NaN,数学,80.0
2,NaN,英语,90.0
3,NaN,NaN,NaN
4,小王,语文,85.0
5,NaN,数学,NaN
6,NaN,英语,90.0
7,NaN,NaN,NaN
8,小刚,语文,85.0
9,NaN,数学,80.0


### 步骤4：删除掉全是空值的行

In [54]:
tmp_data.dropna(axis=0, how='all', inplace=True)

In [55]:
tmp_data

,姓名,科目,分数
0,小明,语文,85.0
1,NaN,数学,80.0
2,NaN,英语,90.0
4,小王,语文,85.0
5,NaN,数学,NaN
6,NaN,英语,90.0
8,小刚,语文,85.0
9,NaN,数学,80.0
10,NaN,英语,90.0


### 步骤5：将分数列为空的填充为0分

In [56]:
tmp_data.fillna({'分数':0})

,姓名,科目,分数
0,小明,语文,85.0
1,NaN,数学,80.0
2,NaN,英语,90.0
4,小王,语文,85.0
5,NaN,数学,0.0
6,NaN,英语,90.0
8,小刚,语文,85.0
9,NaN,数学,80.0
10,NaN,英语,90.0


In [57]:
# 等同于
tmp_data['分数'] = tmp_data['分数'].fillna(0)

In [58]:
tmp_data

,姓名,科目,分数
0,小明,语文,85.0
1,NaN,数学,80.0
2,NaN,英语,90.0
4,小王,语文,85.0
5,NaN,数学,0.0
6,NaN,英语,90.0
8,小刚,语文,85.0
9,NaN,数学,80.0
10,NaN,英语,90.0


### 步骤6：将姓名的缺失值填充

In [59]:
tmp_data['姓名'] = tmp_data['姓名'].fillna(method='ffill')

In [60]:
tmp_data

,姓名,科目,分数
0,小明,语文,85.0
1,小明,数学,80.0
2,小明,英语,90.0
4,小王,语文,85.0
5,小王,数学,0.0
6,小王,英语,90.0
8,小刚,语文,85.0
9,小刚,数学,80.0
10,小刚,英语,90.0


In [61]:
tmp_data.to_excel('./student_excel_clean.xlsx', index=False)

In [62]:
# free memory
del tmp_data

### 12.2 天气数据清洗

In [63]:
df['max_temperature'].fillna(method='ffill', inplace=True)
df['max_temperature'].fillna(method='bfill', inplace=True)
df['max_temperature'].fillna(df['max_temperature'].interpolate(), inplace=True)

In [64]:
df['max_temperature'].isnull()

date
2017-01-01    False
2017-01-02    False
2017-01-03    False
2017-01-04    False
2017-01-05    False
              ...  
2019-12-27    False
2019-12-28    False
2019-12-29    False
2019-12-30    False
2019-12-31    False
Name: max_temperature, Length: 1095, dtype: bool

In [65]:
df[df['max_temperature'].isnull()]

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status,dif,temp_type,temp_type2,wencha_type
date,,,,,,,,,,,


## 13、排序

## Pandas数据排序

Series的排序：
***Series.sort_values(ascending=True, inplace=False)\***
参数说明：

- ascending：默认为True升序排序，为False降序排序
- inplace：是否修改原始Series

DataFrame的排序：
***DataFrame.sort_values(by, ascending=True, inplace=False)\***
参数说明：

- by：字符串或者List<字符串>，单列排序或者多列排序

- ascending：bool或者List

- inplace：是否修改原始DataFramePandas数据排序

  Series的排序：
  ***Series.sort_values(ascending=True, inplace=False)\***
  参数说明：

  - ascending：默认为True升序排序，为False降序排序
  - inplace：是否修改原始Series

  DataFrame的排序：
  ***DataFrame.sort_values(by, ascending=True, inplace=False)\***
  参数说明：

  - by：字符串或者List<字符串>，单列排序或者多列排序
  - ascending：bool或者List
  - inplace：是否修改原始DataFrame

### Series排序

In [66]:
df['aqi'].sort_values()

date
2018-08-19     20
2019-09-12     20
2019-10-05     21
2017-10-04     26
2019-08-02     26
             ... 
2017-01-17    370
2017-01-01    372
2017-02-05    379
2018-01-18    404
2018-11-28    422
Name: aqi, Length: 1095, dtype: int64

In [67]:
df['aqi'].sort_values(ascending=False)


date
2018-11-28    422
2018-01-18    404
2017-02-05    379
2017-01-01    372
2017-01-17    370
             ... 
2019-08-02     26
2017-10-04     26
2019-10-05     21
2019-09-12     20
2018-08-19     20
Name: aqi, Length: 1095, dtype: int64

### DataFrame 排序

In [68]:
# 单列排序
df.sort_values(by='aqi')

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status,dif,temp_type,temp_type2,wencha_type
date,,,,,,,,,,,
2018-08-19,周日,26.0,23.0,小雨,北风4-5级,20,优,3.0,常温,常温,温差小
2019-09-12,周四,22.0,19.0,小雨~阴,东北风2级,20,优,3.0,常温,常温,温差小
2019-10-05,周六,16.0,11.0,小雨,东北风3级,21,优,5.0,常温,常温,温差小
2017-10-04,周三,13.0,11.0,小雨~阴,无持续风向微风,26,优,2.0,常温,常温,温差小
2019-08-02,周五,30.0,24.0,小雨,东北风2级,26,优,6.0,常温,常温,温差小
...,...,...,...,...,...,...,...,...,...,...,...
2017-01-17,周二,8.0,-2.0,多云~阴,无持续风向微风,370,严重,10.0,常温,常温,温差小
2017-01-01,周日,9.0,NaN,多云,无持续风向微风,372,严重,NaN,常温,常温,
2017-02-05,周日,5.0,NaN,晴~多云,无持续风向微风,379,严重,NaN,常温,常温,


In [69]:
# 单列排序
df.sort_values(by='aqi', ascending=False)

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status,dif,temp_type,temp_type2,wencha_type
date,,,,,,,,,,,
2018-11-28,周三,16.0,4.0,晴,西南风1级,422,严重,12.0,常温,常温,温差大
2018-01-18,周四,9.0,-1.0,多云,北风1-2级,404,严重,10.0,常温,常温,温差小
2017-02-05,周日,5.0,NaN,晴~多云,无持续风向微风,379,严重,NaN,常温,常温,
2017-01-01,周日,9.0,NaN,多云,无持续风向微风,372,严重,NaN,常温,常温,
2017-01-17,周二,8.0,-2.0,多云~阴,无持续风向微风,370,严重,10.0,常温,常温,温差小
...,...,...,...,...,...,...,...,...,...,...,...
2019-09-15,周日,19.0,17.0,中雨~小雨,西北风2级,26,优,2.0,常温,常温,温差小
2017-10-04,周三,13.0,11.0,小雨~阴,无持续风向微风,26,优,2.0,常温,常温,温差小
2019-10-05,周六,16.0,11.0,小雨,东北风3级,21,优,5.0,常温,常温,温差小


### 多列排序

In [70]:
# 按空气质量等级、最高温度排序，默认升序
df.sort_values(by=['aqi', 'max_temperature'])

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status,dif,temp_type,temp_type2,wencha_type
date,,,,,,,,,,,
2019-09-12,周四,22.0,19.0,小雨~阴,东北风2级,20,优,3.0,常温,常温,温差小
2018-08-19,周日,26.0,23.0,小雨,北风4-5级,20,优,3.0,常温,常温,温差小
2019-10-05,周六,16.0,11.0,小雨,东北风3级,21,优,5.0,常温,常温,温差小
2017-10-04,周三,13.0,11.0,小雨~阴,无持续风向微风,26,优,2.0,常温,常温,温差小
2019-09-15,周日,19.0,17.0,中雨~小雨,西北风2级,26,优,2.0,常温,常温,温差小
...,...,...,...,...,...,...,...,...,...,...,...
2017-01-17,周二,8.0,-2.0,多云~阴,无持续风向微风,370,严重,10.0,常温,常温,温差小
2017-01-01,周日,9.0,NaN,多云,无持续风向微风,372,严重,NaN,常温,常温,
2017-02-05,周日,5.0,NaN,晴~多云,无持续风向微风,379,严重,NaN,常温,常温,


In [71]:
# 两个字段都是降序
df.sort_values(by=['aqi', 'max_temperature'], ascending=False)

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status,dif,temp_type,temp_type2,wencha_type
date,,,,,,,,,,,
2018-11-28,周三,16.0,4.0,晴,西南风1级,422,严重,12.0,常温,常温,温差大
2018-01-18,周四,9.0,-1.0,多云,北风1-2级,404,严重,10.0,常温,常温,温差小
2017-02-05,周日,5.0,NaN,晴~多云,无持续风向微风,379,严重,NaN,常温,常温,
2017-01-01,周日,9.0,NaN,多云,无持续风向微风,372,严重,NaN,常温,常温,
2017-01-17,周二,8.0,-2.0,多云~阴,无持续风向微风,370,严重,10.0,常温,常温,温差小
...,...,...,...,...,...,...,...,...,...,...,...
2019-09-15,周日,19.0,17.0,中雨~小雨,西北风2级,26,优,2.0,常温,常温,温差小
2017-10-04,周三,13.0,11.0,小雨~阴,无持续风向微风,26,优,2.0,常温,常温,温差小
2019-10-05,周六,16.0,11.0,小雨,东北风3级,21,优,5.0,常温,常温,温差小


In [72]:
# 分别指定升序和降序
df.sort_values(by=['aqi', 'max_temperature'], ascending=[True, False])

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status,dif,temp_type,temp_type2,wencha_type
date,,,,,,,,,,,
2018-08-19,周日,26.0,23.0,小雨,北风4-5级,20,优,3.0,常温,常温,温差小
2019-09-12,周四,22.0,19.0,小雨~阴,东北风2级,20,优,3.0,常温,常温,温差小
2019-10-05,周六,16.0,11.0,小雨,东北风3级,21,优,5.0,常温,常温,温差小
2019-08-02,周五,30.0,24.0,小雨,东北风2级,26,优,6.0,常温,常温,温差小
2019-09-15,周日,19.0,17.0,中雨~小雨,西北风2级,26,优,2.0,常温,常温,温差小
...,...,...,...,...,...,...,...,...,...,...,...
2017-01-17,周二,8.0,-2.0,多云~阴,无持续风向微风,370,严重,10.0,常温,常温,温差小
2017-01-01,周日,9.0,NaN,多云,无持续风向微风,372,严重,NaN,常温,常温,
2017-02-05,周日,5.0,NaN,晴~多云,无持续风向微风,379,严重,NaN,常温,常温,


## 14、字符处理
## Pandas字符串处理

前面我们已经使用了字符串的处理函数：
df["bWendu"].str.replace("℃", "").astype('int32')

***Pandas的字符串处理：\***

1. 使用方法：先获取Series的str属性，然后在属性上调用函数；
2. 只能在字符串列上使用，不能数字列上使用；
3. Dataframe上没有str属性和处理方法
4. Series.str并不是Python原生字符串，而是自己的一套方法，不过大部分和原生str很相似；

***Series.str字符串方法列表参考文档:\***
https://pandas.pydata.org/pandas-docs/stable/reference/series.html#string-handling

***本节演示内容：\***

1. 获取Series的str属性，然后使用各种字符串处理函数
2. 使用str的startswith、contains等bool类Series可以做条件查询
3. 需要多次str处理的链式操作
4. 使用正则表达式的处理

In [73]:
# 判断是不是数字
df["week"].str.isnumeric()

date
2017-01-01    False
2017-01-02    False
2017-01-03    False
2017-01-04    False
2017-01-05    False
              ...  
2019-12-27    False
2019-12-28    False
2019-12-29    False
2019-12-30    False
2019-12-31    False
Name: week, Length: 1095, dtype: bool

In [74]:
# 长度
df["week"].str.len()

date
2017-01-01    2
2017-01-02    2
2017-01-03    2
2017-01-04    2
2017-01-05    2
             ..
2019-12-27    2
2019-12-28    2
2019-12-29    2
2019-12-30    2
2019-12-31    2
Name: week, Length: 1095, dtype: int64

In [75]:
condition = df.index.str.startswith('2019-12')

In [76]:
df[condition]

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status,dif,temp_type,temp_type2,wencha_type
date,,,,,,,,,,,
2019-12-01,周日,9.0,NaN,多云~晴,西北风3级,116,轻度,NaN,常温,常温,
2019-12-02,周一,8.0,1.0,晴,西北风3级,51,良,7.0,常温,常温,温差小
2019-12-03,周二,13.0,3.0,晴,西北风3级,51,良,10.0,常温,常温,温差小
2019-12-04,周三,13.0,2.0,晴,西北风3级,55,良,11.0,常温,常温,温差大
2019-12-05,周四,9.0,-3.0,多云~晴,东北风3级,92,良,12.0,常温,常温,温差大
2019-12-06,周五,8.0,-2.0,晴,东南风2级,112,轻度,10.0,常温,常温,温差小
2019-12-07,周六,8.0,NaN,晴~多云,东南风1级,210,重度,NaN,常温,常温,
2019-12-08,周日,14.0,-1.0,晴,西南风3级,196,中度,15.0,常温,常温,温差大
2019-12-09,周一,16.0,2.0,晴,西风3级,74,良,14.0,常温,常温,温差大


In [77]:
df.index.str.replace('-', '').str[0:6]
df['date'] = df.index


In [78]:
df['date']


date
2017-01-01    2017-01-01
2017-01-02    2017-01-02
2017-01-03    2017-01-03
2017-01-04    2017-01-04
2017-01-05    2017-01-05
                 ...    
2019-12-27    2019-12-27
2019-12-28    2019-12-28
2019-12-29    2019-12-29
2019-12-30    2019-12-30
2019-12-31    2019-12-31
Name: date, Length: 1095, dtype: object

In [79]:
# 正则
# 添加新列
def get_nianyueri(df):
    year,month,day = df['date'].split("-")
    return f"{year}年{month}月{day}日"
df["中文日期"] = df.apply(get_nianyueri, axis=1)

In [80]:
df['中文日期']

date
2017-01-01    2017年01月01日
2017-01-02    2017年01月02日
2017-01-03    2017年01月03日
2017-01-04    2017年01月04日
2017-01-05    2017年01月05日
                 ...     
2019-12-27    2019年12月27日
2019-12-28    2019年12月28日
2019-12-29    2019年12月29日
2019-12-30    2019年12月30日
2019-12-31    2019年12月31日
Name: 中文日期, Length: 1095, dtype: object

In [81]:
# 问题：怎样将“2018年12月31日”中的年、月、日三个中文字符去除？
# 方法1：链式replace
df["中文日期"].str.replace("年", "").str.replace("月","").str.replace("日", "")

date
2017-01-01    20170101
2017-01-02    20170102
2017-01-03    20170103
2017-01-04    20170104
2017-01-05    20170105
                ...   
2019-12-27    20191227
2019-12-28    20191228
2019-12-29    20191229
2019-12-30    20191230
2019-12-31    20191231
Name: 中文日期, Length: 1095, dtype: object

In [82]:
# 方法2：正则表达式替换
df["中文日期"].str.replace("[年月日]", "")

date
2017-01-01    20170101
2017-01-02    20170102
2017-01-03    20170103
2017-01-04    20170104
2017-01-05    20170105
                ...   
2019-12-27    20191227
2019-12-28    20191228
2019-12-29    20191229
2019-12-30    20191230
2019-12-31    20191231
Name: 中文日期, Length: 1095, dtype: object

In [83]:
%timeit df["中文日期"].str.replace("[年月日]", "")

1.49 ms ± 65.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## 15、index

##  Pandas的索引index的用途
把数据存储于普通的column列也能用于数据查询，那使用index有什么好处？

index的用途总结：

1. 更方便的数据查询；
2. 使用index可以获得性能提升；
3. 自动的数据对齐功能；
4. 更多更强大的数据结构支持；

In [84]:
tmp_data = pd.read_csv('./ratings.csv')

In [85]:
tmp_data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [86]:
tmp_data.count()

userId       100836
movieId      100836
rating       100836
timestamp    100836
dtype: int64

In [87]:
# index查询数据
# drop==False，让索引列还保持在column
tmp_data.set_index("userId", inplace=True, drop=False)

In [88]:
tmp_data.head()


,userId,movieId,rating,timestamp
userId,,,,
1,1,1,4.0,964982703
1,1,3,4.0,964981247
1,1,6,4.0,964982224
1,1,47,5.0,964983815
1,1,50,5.0,964982931


In [89]:
tmp_data.index


Int64Index([  1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
            ...
            610, 610, 610, 610, 610, 610, 610, 610, 610, 610],
           dtype='int64', name='userId', length=100836)

In [90]:
# 使用index的查询方法
tmp_data.loc[500].head(5)

,userId,movieId,rating,timestamp
userId,,,,
500,500,1,4.0,1005527755
500,500,11,1.0,1005528017
500,500,39,1.0,1005527926
500,500,101,1.0,1005527980
500,500,104,4.0,1005528065


In [91]:
# 使用column的condition查询方法
tmp_data.loc[tmp_data["userId"] == 500].head()


,userId,movieId,rating,timestamp
userId,,,,
500,500,1,4.0,1005527755
500,500,11,1.0,1005528017
500,500,39,1.0,1005527926
500,500,101,1.0,1005527980
500,500,104,4.0,1005528065


### 使用index会提升查询性能

- 如果index是唯一的，Pandas会使用哈希表优化，查询性能为O(1);
- 如果index不是唯一的，但是有序，Pandas会使用二分查找算法，查询性能为O(logN);
- 如果index是完全随机的，那么每次查询都要扫描全表，查询性能为O(N);

In [92]:
# 将数据随机打散
from sklearn.utils import shuffle
df_shuffle = shuffle(tmp_data)

In [93]:
df_shuffle.head()

,userId,movieId,rating,timestamp
userId,,,,
600,600,1034,2.5,1237760659
167,167,1278,4.0,1154718813
560,560,37733,4.5,1469650578
448,448,118814,2.0,1441045203
608,608,7325,4.0,1117506234


In [94]:
# 索引是否是递增的
df_shuffle.index.is_monotonic_increasing

False

In [95]:
df_shuffle.index.is_unique

False

In [96]:
# 计时，查询id==500数据性能
%timeit df_shuffle.loc[500]

298 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### 将index排序后的查询

In [97]:
df_sorted = df_shuffle.sort_index()

In [98]:
df_sorted.head()

,userId,movieId,rating,timestamp
userId,,,,
1,1,2193,4.0,964981710
1,1,1396,3.0,964983017
1,1,2174,4.0,964981680
1,1,1270,5.0,964983705
1,1,3273,5.0,964983536


In [99]:
# 索引是否是递增的
df_sorted.index.is_monotonic_increasing

True

In [100]:
df_sorted.index.is_unique

False

In [101]:
%timeit df_sorted.loc[500]

52.5 µs ± 5.09 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### 使用index能自动对齐数据
包括series和dataframe

In [102]:
s1 = pd.Series([1,2,3], index=list("abc"))

In [103]:
s1

a    1
b    2
c    3
dtype: int64

In [104]:
s2 = pd.Series([2,3,4], index=list("bcd"))

In [105]:
s2

b    2
c    3
d    4
dtype: int64

In [106]:
s1+s2

a    NaN
b    4.0
c    6.0
d    NaN
dtype: float64

In [107]:
# del
del s1
del s2
del df_sorted
del df_shuffle
del tmp_data

## 16、Pandas怎样实现groupby分组统计

类似SQL：
select city,max(temperature) from city_weather group by city;

groupby：先对数据分组，然后在每个分组上应用聚合函数、转换函数

本次演示：


一、分组使用聚合函数做数据统计

二、遍历groupby的结果理解执行流程


三、实例分组探索天气数据

In [108]:
# 加上这一句，能在jupyter notebook展示matplot图表
# %matplotlib inline
%matplotlib widget

In [109]:
test_df = pd.DataFrame({'A': ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],
                   'B': ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
                   'C': np.random.randn(8),
                   'D': np.random.randn(8)})
test_df

,A,B,C,D
0,foo,one,-1.097629,-0.855014
1,bar,one,0.610262,0.004293
2,foo,two,-0.116379,1.009069
3,bar,three,0.338883,-0.851371
4,foo,two,-2.534381,1.065644
5,bar,two,-0.396466,1.061021
6,foo,one,-0.750336,0.946112
7,foo,three,-0.430926,1.694233


###  一、分组使用聚合函数做数据统计

In [110]:
# 1、单个列groupby，查询所有数据列的统计
test_df.groupby('A').sum()


,C,D
A,,
bar,0.552679,0.213943
foo,-4.929652,3.860044


我们看到：

1. groupby中的'A'变成了数据的索引列
2. 因为要统计sum，但B列不是数字，所以被自动忽略掉

In [111]:
# 2、多个列groupby，查询所有数据列的统计
test_df.groupby(['A','B']).mean()


C         D
A   B                        
bar one    0.610262  0.004293
    three  0.338883 -0.851371
    two   -0.396466  1.061021
foo one   -0.923983  0.045549
    three -0.430926  1.694233
    two   -1.325380  1.037356

我们看到：('A','B')成对变成了二级索引

In [112]:
test_df.groupby(['A','B'], as_index=False).mean()

,A,B,C,D
0,bar,one,0.610262,0.004293
1,bar,three,0.338883,-0.851371
2,bar,two,-0.396466,1.061021
3,foo,one,-0.923983,0.045549
4,foo,three,-0.430926,1.694233
5,foo,two,-1.325380,1.037356


In [113]:
# 3、同时查看多种数据统计
test_df.groupby('A').agg([np.sum, np.mean, np.std])

C                             D                    
          sum      mean       std       sum      mean       std
A                                                              
bar  0.552679  0.184226  0.520879  0.213943  0.071314  0.957956
foo -4.929652 -0.985930  0.939392  3.860044  0.772009  0.957918

In [114]:
# 4、查看单列的结果数据统计

# 方法1：预过滤，性能更好
test_df.groupby('A')['C'].agg([np.sum, np.mean, np.std])


,sum,mean,std
A,,,
bar,0.552679,0.184226,0.520879
foo,-4.929652,-0.985930,0.939392


In [115]:
# 方法2
test_df.groupby('A').agg([np.sum, np.mean, np.std])['C']

,sum,mean,std
A,,,
bar,0.552679,0.184226,0.520879
foo,-4.929652,-0.985930,0.939392


In [116]:
# 5、不同列使用不同的聚合函数
test_df.groupby('A').agg({"C":np.sum, "D":np.mean})

,C,D
A,,
bar,0.552679,0.071314
foo,-4.929652,0.772009


### 二、遍历groupby的结果理解执行流程
for循环可以直接遍历每个group

In [117]:
g = test_df.groupby('A')
g

In [118]:
for name, group in g:
    print(name)
    print(group)
    print('-' * 40)

bar
     A      B         C         D
1  bar    one  0.610262  0.004293
3  bar  three  0.338883 -0.851371
5  bar    two -0.396466  1.061021
----------------------------------------
foo
     A      B         C         D
0  foo    one -1.097629 -0.855014
2  foo    two -0.116379  1.009069
4  foo    two -2.534381  1.065644
6  foo    one -0.750336  0.946112
7  foo  three -0.430926  1.694233
----------------------------------------


In [119]:
# 可以获取单个分组的数据
g.get_group('bar')

,A,B,C,D
1,bar,one,0.610262,0.004293
3,bar,three,0.338883,-0.851371
5,bar,two,-0.396466,1.061021


In [120]:
# 遍历多个列聚合的分组
g = test_df.groupby(['A', 'B'])

In [121]:
for name, group in g:
    print(name)
    print(group)
    print('-' * 40)

('bar', 'one')
     A    B         C         D
1  bar  one  0.610262  0.004293
----------------------------------------
('bar', 'three')
     A      B         C         D
3  bar  three  0.338883 -0.851371
----------------------------------------
('bar', 'two')
     A    B         C         D
5  bar  two -0.396466  1.061021
----------------------------------------
('foo', 'one')
     A    B         C         D
0  foo  one -1.097629 -0.855014
6  foo  one -0.750336  0.946112
----------------------------------------
('foo', 'three')
     A      B         C         D
7  foo  three -0.430926  1.694233
----------------------------------------
('foo', 'two')
     A    B         C         D
2  foo  two -0.116379  1.009069
4  foo  two -2.534381  1.065644
----------------------------------------


In [122]:
# 可以看到，name是一个2个元素的tuple，代表不同的列
g.get_group(('foo', 'one'))

,A,B,C,D
0,foo,one,-1.097629,-0.855014
6,foo,one,-0.750336,0.946112


In [123]:
# 可以直接查询group后的某几列，生成Series或者子DataFrame
g['C']

In [124]:
for name, group in g['C']:
    print(name)
    print(group)
    print(type(group))
    print('-' * 40)
del g
del test_df

('bar', 'one')
1    0.610262
Name: C, dtype: float64
<class 'pandas.core.series.Series'>
----------------------------------------
('bar', 'three')
3    0.338883
Name: C, dtype: float64
<class 'pandas.core.series.Series'>
----------------------------------------
('bar', 'two')
5   -0.396466
Name: C, dtype: float64
<class 'pandas.core.series.Series'>
----------------------------------------
('foo', 'one')
0   -1.097629
6   -0.750336
Name: C, dtype: float64
<class 'pandas.core.series.Series'>
----------------------------------------
('foo', 'three')
7   -0.430926
Name: C, dtype: float64
<class 'pandas.core.series.Series'>
----------------------------------------
('foo', 'two')
2   -0.116379
4   -2.534381
Name: C, dtype: float64
<class 'pandas.core.series.Series'>
----------------------------------------


### 三、实例分组探索天气数据

In [125]:
df.head()

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status,dif,temp_type,temp_type2,wencha_type,date,中文日期
date,,,,,,,,,,,,,
2017-01-01,周日,9.0,NaN,多云,无持续风向微风,372,严重,NaN,常温,常温,,2017-01-01,2017年01月01日
2017-01-02,周一,9.0,NaN,霾,无持续风向微风,361,严重,NaN,常温,常温,,2017-01-02,2017年01月02日
2017-01-03,周二,9.0,NaN,霾~雾,无持续风向微风,280,重度,NaN,常温,常温,,2017-01-03,2017年01月03日
2017-01-04,周三,9.0,2.0,小雨,无持续风向微风,193,中度,7.0,常温,常温,温差小,2017-01-04,2017年01月04日
2017-01-05,周四,5.0,1.0,小雨,无持续风向微风,216,重度,4.0,常温,常温,温差小,2017-01-05,2017年01月05日


In [126]:
df['month'] = df.index.str[:7]

In [127]:
df.head()

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status,dif,temp_type,temp_type2,wencha_type,date,中文日期,month
date,,,,,,,,,,,,,,
2017-01-01,周日,9.0,NaN,多云,无持续风向微风,372,严重,NaN,常温,常温,,2017-01-01,2017年01月01日,2017-01
2017-01-02,周一,9.0,NaN,霾,无持续风向微风,361,严重,NaN,常温,常温,,2017-01-02,2017年01月02日,2017-01
2017-01-03,周二,9.0,NaN,霾~雾,无持续风向微风,280,重度,NaN,常温,常温,,2017-01-03,2017年01月03日,2017-01
2017-01-04,周三,9.0,2.0,小雨,无持续风向微风,193,中度,7.0,常温,常温,温差小,2017-01-04,2017年01月04日,2017-01
2017-01-05,周四,5.0,1.0,小雨,无持续风向微风,216,重度,4.0,常温,常温,温差小,2017-01-05,2017年01月05日,2017-01


In [128]:
# 查看每个月的最高温度
data = df.groupby('month')['max_temperature'].max()


In [129]:
data.head(12)

month
2017-01    14.0
2017-02    21.0
2017-03    21.0
2017-04    34.0
2017-05    39.0
2017-06    38.0
2017-07    38.0
2017-08    37.0
2017-09    33.0
2017-10    25.0
2017-11    24.0
2017-12    14.0
Name: max_temperature, dtype: float64

In [130]:
type(data)

pandas.core.series.Series

In [131]:

data[0:12].plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

d:\python\lib\site-packages\pandas\plotting\_matplotlib\core.py:1235: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(xticklabels)


<AxesSubplot:xlabel='month'>

In [132]:
# 查看每个月的最高温度、最低温度、平均空气质量指数
df.head()

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status,dif,temp_type,temp_type2,wencha_type,date,中文日期,month
date,,,,,,,,,,,,,,
2017-01-01,周日,9.0,NaN,多云,无持续风向微风,372,严重,NaN,常温,常温,,2017-01-01,2017年01月01日,2017-01
2017-01-02,周一,9.0,NaN,霾,无持续风向微风,361,严重,NaN,常温,常温,,2017-01-02,2017年01月02日,2017-01
2017-01-03,周二,9.0,NaN,霾~雾,无持续风向微风,280,重度,NaN,常温,常温,,2017-01-03,2017年01月03日,2017-01
2017-01-04,周三,9.0,2.0,小雨,无持续风向微风,193,中度,7.0,常温,常温,温差小,2017-01-04,2017年01月04日,2017-01
2017-01-05,周四,5.0,1.0,小雨,无持续风向微风,216,重度,4.0,常温,常温,温差小,2017-01-05,2017年01月05日,2017-01


In [133]:
group_data = df.groupby('month').agg({"max_temperature":np.max, "min_temperature":np.min, "aqi":np.mean})
group_data

,max_temperature,min_temperature,aqi
month,,,
2017-01,14.0,-5.0,201.612903
2017-02,21.0,-6.0,150.321429
2017-03,21.0,-1.0,118.064516
2017-04,34.0,7.0,103.033333
2017-05,39.0,13.0,118.258065
2017-06,38.0,13.0,91.900000
2017-07,38.0,21.0,73.419355
2017-08,37.0,16.0,72.129032
2017-09,33.0,15.0,79.566667


In [134]:
group_data.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

d:\python\lib\site-packages\pandas\plotting\_matplotlib\core.py:1235: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(xticklabels)


<AxesSubplot:xlabel='month'>

### 进阶

In [135]:
df['month'] = df.index.str[5:7]


In [136]:
df.head()

,week,max_temperature,min_temperature,day_status,wind,aqi,aqi_status,dif,temp_type,temp_type2,wencha_type,date,中文日期,month
date,,,,,,,,,,,,,,
2017-01-01,周日,9.0,NaN,多云,无持续风向微风,372,严重,NaN,常温,常温,,2017-01-01,2017年01月01日,01
2017-01-02,周一,9.0,NaN,霾,无持续风向微风,361,严重,NaN,常温,常温,,2017-01-02,2017年01月02日,01
2017-01-03,周二,9.0,NaN,霾~雾,无持续风向微风,280,重度,NaN,常温,常温,,2017-01-03,2017年01月03日,01
2017-01-04,周三,9.0,2.0,小雨,无持续风向微风,193,中度,7.0,常温,常温,温差小,2017-01-04,2017年01月04日,01
2017-01-05,周四,5.0,1.0,小雨,无持续风向微风,216,重度,4.0,常温,常温,温差小,2017-01-05,2017年01月05日,01


In [137]:
data = df.groupby('month')[['max_temperature', 'min_temperature']].max()
data

,max_temperature,min_temperature
month,,
01,14.0,2.0
02,21.0,6.0
03,31.0,15.0
04,34.0,19.0
05,39.0,25.0
06,39.0,26.0
07,39.0,29.0
08,37.0,27.0
09,36.0,25.0


In [138]:
data.plot(xlabel='month', xticks=[i for i in range(1,13)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='month'>

In [139]:
group_data = df.groupby('month').agg({"max_temperature":np.max, "min_temperature":np.min, "aqi":np.mean})
group_data

,max_temperature,min_temperature,aqi
month,,,
01,14.0,-10.0,172.430108
02,21.0,-6.0,145.095238
03,31.0,-1.0,113.462366
04,34.0,3.0,105.933333
05,39.0,13.0,98.408602
06,39.0,13.0,81.755556
07,39.0,21.0,67.623656
08,37.0,16.0,64.763441
09,36.0,12.0,68.266667


In [140]:
group_data.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

d:\python\lib\site-packages\pandas\plotting\_matplotlib\core.py:1235: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(xticklabels)


<AxesSubplot:xlabel='month'>

In [141]:
del data
del df